# Convert a pandas dataframe to geojson for web-mapping

Author: Geoff Boeing

Original: [pandas-to-geojson](https://github.com/gboeing/urban-data-science/blob/dc86c9c89b73f87f97301883d7456f1f814589f5/17-Leaflet-Web-Mapping/pandas-to-geojson.ipynb)

In [ ]:
import json
import requests
import pandas as pd

First download data from the city of Berkeley's API. You can use Socrata's `$limit` parameter to specify how many rows to fetch.

In [ ]:
endpoint_url = "https://data.cityofberkeley.info/resource/bscu-qpbu.json?$limit=20&$where=latitude%20%3C%3E%20%22%22"

In [ ]:
response = requests.get(endpoint_url)
data = response.json()

In [ ]:
df = pd.DataFrame(data)
df["latitude"] = df["latitude"].astype(float)
df["longitude"] = df["longitude"].astype(float)
df["street_address"] = df["street_address"].str.title()

In [ ]:
cols = ["request_detail", "request_subcategory", "latitude", "longitude", "street_address", "case_status"]
df_subset = df[cols]
df_geo = df_subset.dropna(subset=["latitude", "longitude"], axis=0, inplace=False)

In [ ]:
def df_to_geojson(df, properties, lat="latitude", lon="longitude"):
    geojson = {"type": "FeatureCollection", "features": []}

    for _, row in df.iterrows():
        feature = {
            "type": "Feature",
            "properties": {},
            "geometry": {"type": "Point", "coordinates": []},
        }
        feature["geometry"]["coordinates"] = [row[lon], row[lat]]
        for prop in properties:
            feature["properties"][prop] = row[prop]
        geojson["features"].append(feature)

    return geojson

In [ ]:
cols = ["street_address", "request_detail", "request_subcategory", "case_status"]
geojson = df_to_geojson(df_geo, cols)
geojson["features"][0]

In nteract, this geojson can be rendered directly with the built-in leaflet renderer.

In [ ]:
import IPython

IPython.display.display({"application/geo+json": geojson}, raw=True)